<a href="https://colab.research.google.com/github/Aditi0102/ML-notebook/blob/main/AiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [1]:
%tensorflow_version 2.x

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import cv2
from tensorflow.keras.optimizers import RMSprop
from google.colab.patches import cv2_imshow
import math

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aditiagarwal01","key":"4c56849565c39183e765f285aeafe7d3"}\n'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

100% 3.57G/3.58G [01:29<00:00, 33.8MB/s]
100% 3.58G/3.58G [01:29<00:00, 42.8MB/s]


In [6]:
!unzip \*.zip  && rm *.zip
clear_output()
print("Extracted")

Extracted


In [7]:
import os
train_folder = 'Directories/training/violent'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

In [8]:
# Training Data

count = 0
for i in range (1,300):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (train_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [9]:
import os
test_folder = 'Directories/validation/violent'
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

In [10]:
# Testing Data
count = 0
for i in range (300,400):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [11]:
import os
nv_folder = 'Directories/training/non_violent'
if not os.path.exists(nv_folder):
    os.makedirs(nv_folder)

In [12]:
count = 0
for i in range (1,300):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")

Done Training with Non-Violence !!


In [13]:
import os
nv_test_folder = 'Directories/validation/non_violent'
if not os.path.exists(nv_test_folder):
    os.makedirs(nv_test_folder)

In [14]:
count = 0
for i in range (300,400):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")


Done Training with Non-Violence !!


In [29]:

base_dir = './Directories'
train_dir = os.path.join(base_dir,'training')
validation_dir = os.path.join(base_dir,'validation')

In [30]:
train_violent_dir = os.path.join(train_dir,'violent')
train_nonviolent_dir=os.path.join(train_dir,'non_violent')
validation_violent_dir=os.path.join(validation_dir,'violent')
validation_nonviolent_dir=os.path.join(validation_dir,'non_violent')


In [41]:
train_datagen= ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale =1./255)

In [42]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (64,64),batch_size=20, class_mode='binary')

Found 81349 images belonging to 2 classes.


In [43]:
validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(64,64),batch_size=20,class_mode='binary')


Found 28853 images belonging to 2 classes.


In [44]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [45]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=1e-4),metrics=['accuracy'])



In [46]:
model1= model.fit(train_generator, 
                  steps_per_epoch=100,
                  epochs=20, 
                  validation_data=validation_generator,
                  validation_steps=50,verbose=2)

Epoch 1/20
100/100 - 4s - loss: 0.6635 - accuracy: 0.5990 - val_loss: 0.6506 - val_accuracy: 0.6250
Epoch 2/20
100/100 - 3s - loss: 0.5954 - accuracy: 0.6770 - val_loss: 0.6815 - val_accuracy: 0.6250
Epoch 3/20
100/100 - 3s - loss: 0.5674 - accuracy: 0.7025 - val_loss: 0.8717 - val_accuracy: 0.4690
Epoch 4/20
100/100 - 3s - loss: 0.5072 - accuracy: 0.7570 - val_loss: 1.1673 - val_accuracy: 0.3970
Epoch 5/20
100/100 - 3s - loss: 0.4864 - accuracy: 0.7580 - val_loss: 0.7109 - val_accuracy: 0.6210
Epoch 6/20
100/100 - 3s - loss: 0.4704 - accuracy: 0.7645 - val_loss: 1.1030 - val_accuracy: 0.4500
Epoch 7/20
100/100 - 3s - loss: 0.4338 - accuracy: 0.7915 - val_loss: 0.9029 - val_accuracy: 0.5480
Epoch 8/20
100/100 - 3s - loss: 0.4301 - accuracy: 0.7880 - val_loss: 0.9314 - val_accuracy: 0.5480
Epoch 9/20
100/100 - 3s - loss: 0.3937 - accuracy: 0.8215 - val_loss: 0.8372 - val_accuracy: 0.6170
Epoch 10/20
100/100 - 3s - loss: 0.3987 - accuracy: 0.8145 - val_loss: 0.6599 - val_accuracy: 0.6730

In [47]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1611162302.h5
